In [19]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
from PIL import Image, ImageTk
import os
import sys
import time

class PAUObjectDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("PAU Location Object Detector")
        self.root.geometry("1200x800")
        self.root.configure(bg="#f0f0f0")
        
        # Default video paths
        self.default_videos = [
            r"C:\Users\maxos\CSC418\dmoru883-dmoruCSC418\Week 5\Videos 1-3\Video 1.mp4",
            r"C:\Users\maxos\CSC418\dmoru883-dmoruCSC418\Week 5\Videos 1-3\Video 2.mp4",
            r"C:\Users\maxos\CSC418\dmoru883-dmoruCSC418\Week 5\Videos 1-3\Video 3.mp4"
        ]
        
        # Correct YOLO and Haar cascade paths
        self.yolo_base_path = r"C:\Users\maxos\CSC418\dmoru883-dmoruCSC418\Week 5\cfg\cfg"
        self.weights_path = os.path.join(self.yolo_base_path, "yolov3.weights")
        self.cfg_path = os.path.join(self.yolo_base_path, "yolov3.cfg")
        self.names_path = os.path.join(self.yolo_base_path, "coco.names")
        self.face_cascade_path = r"C:\Users\maxos\CSC418\dmoru883-dmoruCSC418\Week 5\haarcascade_frontalface_default.xml"
        
        # Video handling variables
        self.video_paths = []
        self.current_video_index = 0
        self.video_capture = None
        self.is_playing = False
        self.detection_enabled = True
        self.face_detection_enabled = True  # Added face detection toggle
        self.frame_count = 0
        self.detection_frequency = 5
        self.previous_detections = []
        self.last_frame_time = 0
        
        # Initialize detectors
        try:
            self.initialize_yolo()
            self.initialize_face_cascade()
        except Exception as e:
            messagebox.showerror("Initialization Error", 
                               f"Failed to initialize detectors: {str(e)}\n\n"
                               "Please verify all required files are present.")
            sys.exit(1)
        
        # Create GUI elements
        self.create_widgets()
        
        # Try to load default videos
        self.try_load_default_videos()

    def initialize_yolo(self):
        for path in [self.weights_path, self.cfg_path, self.names_path]:
            if not os.path.exists(path):
                raise FileNotFoundError(f"Required YOLO file not found: {path}")
                
        self.net = cv2.dnn.readNet(self.weights_path, self.cfg_path)
        
        if cv2.cuda.getCudaEnabledDeviceCount() > 0:
            try:
                self.net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
                self.net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
                print("Using CUDA backend for YOLO")
            except:
                print("CUDA available but failed to initialize, falling back to CPU")
        else:
            print("CUDA not available, using CPU for YOLO")
            
        with open(self.names_path, 'r') as f:
            self.classes = f.read().splitlines()
            
        self.colors = np.random.uniform(0, 255, size=(len(self.classes), 3))
        self.output_layers = self.net.getUnconnectedOutLayersNames()

    def initialize_face_cascade(self):
        if not os.path.exists(self.face_cascade_path):
            raise FileNotFoundError(f"Haar cascade file not found: {self.face_cascade_path}")
        self.face_cascade = cv2.CascadeClassifier(self.face_cascade_path)
        if self.face_cascade.empty():
            raise ValueError("Failed to load Haar cascade classifier")

    def try_load_default_videos(self):
        existing_videos = [path for path in self.default_videos if os.path.exists(path)]
        if existing_videos:
            self.video_paths = existing_videos
            self.current_video_index = 0
            self.load_video()
            self.enable_control_buttons()
            self.update_info_label()
            self.status_bar.config(text=f"Loaded {len(existing_videos)} default videos")
        else:
            self.status_bar.config(text="Default videos not found. Please select videos manually.")

    def create_widgets(self):
        style = ttk.Style()
        style.configure('TButton', font=('Arial', 10), padding=5)
        style.configure('TFrame', background='#f0f0f0')
        
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        top_frame = ttk.Frame(main_frame, padding="5")
        top_frame.pack(fill=tk.X)
        
        selection_frame = ttk.LabelFrame(top_frame, text="Video Selection", padding="5")
        selection_frame.pack(side=tk.LEFT, padx=5, pady=5, fill=tk.X, expand=True)
        
        self.select_btn = ttk.Button(selection_frame, text="Select Videos", command=self.select_videos)
        self.select_btn.pack(side=tk.LEFT, padx=5)
        
        self.use_default_btn = ttk.Button(selection_frame, text="Use Default Videos", command=self.use_default_videos)
        self.use_default_btn.pack(side=tk.LEFT, padx=5)
        
        control_frame = ttk.LabelFrame(top_frame, text="Video Controls", padding="5")
        control_frame.pack(side=tk.RIGHT, padx=5, pady=5)
        
        self.play_btn = ttk.Button(control_frame, text="Play", command=self.toggle_play, state=tk.DISABLED)
        self.play_btn.pack(side=tk.LEFT, padx=5)
        
        self.stop_btn = ttk.Button(control_frame, text="Stop", command=self.stop_video, state=tk.DISABLED)
        self.stop_btn.pack(side=tk.LEFT, padx=5)
        
        self.next_btn = ttk.Button(control_frame, text="Next Video", command=self.next_video, state=tk.DISABLED)
        self.next_btn.pack(side=tk.LEFT, padx=5)
        
        self.detection_toggle = ttk.Button(control_frame, text="Toggle YOLO", command=self.toggle_detection, state=tk.DISABLED)
        self.detection_toggle.pack(side=tk.LEFT, padx=5)
        
        self.face_toggle = ttk.Button(control_frame, text="Toggle Face", command=self.toggle_face_detection, state=tk.DISABLED)
        self.face_toggle.pack(side=tk.LEFT, padx=5)
        
        display_frame = ttk.Frame(main_frame, padding="5")
        display_frame.pack(fill=tk.BOTH, expand=True, pady=5)
        
        video_container = ttk.LabelFrame(display_frame, text="Video Display", padding="5")
        video_container.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        self.video_frame = ttk.Label(video_container)
        self.video_frame.pack(padx=5, pady=5, fill=tk.BOTH, expand=True)
        
        self.info_label = ttk.Label(video_container, text="No video loaded", font=('Arial', 10))
        self.info_label.pack(pady=5)
        
        results_container = ttk.LabelFrame(display_frame, text="Detection Results", padding="5", width=250)
        results_container.pack(side=tk.RIGHT, fill=tk.Y, padx=5, pady=5)
        results_container.pack_propagate(False)
        
        stats_frame = ttk.Frame(results_container)
        stats_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(stats_frame, text="Detection Statistics:", font=('Arial', 10, 'bold')).pack(anchor=tk.W)
        self.stats_label = ttk.Label(stats_frame, text="No objects detected", font=('Arial', 9))
        self.stats_label.pack(anchor=tk.W, pady=5)
        
        self.results_text = tk.Text(results_container, height=20, width=30, wrap=tk.WORD, font=('Arial', 9))
        self.results_text.pack(fill=tk.BOTH, expand=True, pady=5)
        
        scrollbar = ttk.Scrollbar(results_container, command=self.results_text.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.results_text.config(yscrollcommand=scrollbar.set)
        
        status_frame = ttk.Frame(main_frame, padding="2")
        status_frame.pack(fill=tk.X, side=tk.BOTTOM)
        
        self.status_bar = ttk.Label(status_frame, text="Ready", relief=tk.SUNKEN, anchor=tk.W)
        self.status_bar.pack(fill=tk.X)

    def enable_control_buttons(self):
        self.play_btn.config(state=tk.NORMAL)
        self.stop_btn.config(state=tk.NORMAL)
        self.next_btn.config(state=tk.NORMAL)
        self.detection_toggle.config(state=tk.NORMAL)
        self.face_toggle.config(state=tk.NORMAL)

    def use_default_videos(self):
        existing_videos = [path for path in self.default_videos if os.path.exists(path)]
        if not existing_videos:
            messagebox.showwarning("Warning", "Default videos not found. Please check paths or select custom videos.")
            return
            
        self.video_paths = existing_videos
        self.current_video_index = 0
        self.load_video()
        self.enable_control_buttons()
        self.update_info_label()
        self.status_bar.config(text=f"Loaded {len(existing_videos)} default videos")

    def select_videos(self):
        selected_paths = filedialog.askopenfilenames(
            title="Select PAU Videos",
            filetypes=[("Video Files", "*.mp4 *.avi *.mov *.mkv")]
        )
        
        if not selected_paths:
            return
            
        if len(selected_paths) > 3:
            selected_paths = selected_paths[:3]
            messagebox.showwarning("Warning", "Only the first 3 videos will be used")
        
        self.video_paths = list(selected_paths)
        self.current_video_index = 0
        self.load_video()
        self.enable_control_buttons()
        self.update_info_label()
        self.status_bar.config(text=f"Loaded {len(selected_paths)} custom videos")

    def load_video(self):
        if not self.video_paths:
            return
            
        if self.video_capture:
            self.video_capture.release()
            self.video_capture = None
            
        try:
            video_path = self.video_paths[self.current_video_index]
            self.video_capture = cv2.VideoCapture(video_path)
            if not self.video_capture.isOpened():
                raise Exception(f"Could not open video file: {video_path}")
                
            self.frame_width = int(self.video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
            self.frame_height = int(self.video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
            self.fps = self.video_capture.get(cv2.CAP_PROP_FPS) or 30
            self.total_frames = int(self.video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
            
            self.is_playing = False
            self.frame_count = 0
            self.update_info_label()
            self.show_current_frame()
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load video: {str(e)}")
            if self.video_paths:
                self.video_paths.pop(self.current_video_index)
                self.current_video_index = min(self.current_video_index, len(self.video_paths) - 1)
                if self.video_paths:
                    self.load_video()
                else:
                    self.status_bar.config(text="No valid videos available")
                    self.info_label.config(text="No video loaded")

    def toggle_play(self):
        if not self.video_capture:
            return
            
        self.is_playing = not self.is_playing
        
        if self.is_playing:
            self.play_btn.config(text="Pause")
            self.status_bar.config(text="Playing video...")
            self.last_frame_time = time.time()
            self.play_video()
        else:
            self.play_btn.config(text="Play")
            self.status_bar.config(text="Paused")

    def play_video(self):
        if not self.is_playing or not self.video_capture:
            return
            
        ret, frame = self.video_capture.read()
        
        if not ret:
            self.video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
            self.play_video()
            return
        
        self.frame_count += 1
        detected_objects = {}
        
        if self.detection_enabled and (self.frame_count % self.detection_frequency == 0):
            frame, detected_objects = self.detect_objects(frame)
        elif self.detection_enabled:
            frame = self.draw_previous_detections(frame)
            
        if self.face_detection_enabled:
            frame, faces = self.detect_faces(frame)
            if faces:
                detected_objects['face'] = faces
                
        if detected_objects:
            self.update_detection_results(detected_objects)
        
        display_frame = self.resize_frame_to_fit(frame)
        display_frame = cv2.cvtColor(display_frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(display_frame)
        img = ImageTk.PhotoImage(image=img)
        
        self.video_frame.img = img
        self.video_frame.config(image=img)
        
        current_time = time.time()
        elapsed = (current_time - self.last_frame_time) * 1000
        delay = max(1, int(1000 / self.fps - elapsed))
        self.last_frame_time = current_time
        
        self.root.after(delay, self.play_video)

    def resize_frame_to_fit(self, frame):
        max_width, max_height = 800, 600
        h, w = frame.shape[:2]
        
        if h > max_height or w > max_width:
            scale = min(max_width/w, max_height/h)
            return cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
        return frame

    def detect_objects(self, frame):
        try:
            height, width = frame.shape[:2]
            blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
            self.net.setInput(blob)
            layer_outputs = self.net.forward(self.output_layers)
            
            boxes, confidences, class_ids = [], [], []
            
            for output in layer_outputs:
                for detection in output:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    
                    if confidence > 0.5:
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)
                        
                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)
            
            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
            detected_objects = {}
            self.previous_detections = []
            
            if len(indexes) > 0:
                for i in indexes.flatten():
                    x, y, w, h = boxes[i]
                    x = max(0, min(x, width - 1))
                    y = max(0, min(y, height - 1))
                    w = min(w, width - x)
                    h = min(h, height - y)
                    
                    label = str(self.classes[class_ids[i]])
                    confidence = round(confidences[i] * 100, 1)
                    color = tuple(self.colors[class_ids[i]].tolist())
                    
                    self.previous_detections.append({
                        'box': [x, y, w, h],
                        'label': label,
                        'confidence': confidence,
                        'color': color
                    })
                    
                    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                    text_width = len(label) * 13 + 65
                    cv2.rectangle(frame, (x, y - 30), (x + text_width, y), color, -1)
                    cv2.putText(frame, f"{label} {confidence}%", (x, y - 10), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    
                    detected_objects[label] = detected_objects.get(label, 0) + 1
            
            return frame, detected_objects
        except Exception as e:
            print(f"YOLO detection error: {str(e)}")
            return frame, {}

    def detect_faces(self, frame):
        try:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(
                gray,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(30, 30)
            )
            
            face_count = 0
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, "Face", (x, y - 10), 
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                face_count += 1
            
            return frame, face_count
        except Exception as e:
            print(f"Face detection error: {str(e)}")
            return frame, 0

    def draw_previous_detections(self, frame):
        for detection in self.previous_detections:
            x, y, w, h = detection['box']
            label = detection['label']
            confidence = detection['confidence']
            color = detection['color']
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text_width = len(label) * 13 + 65
            cv2.rectangle(frame, (x, y - 30), (x + text_width, y), color, -1)
            cv2.putText(frame, f"{label} {confidence}%", (x, y - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        return frame

    def update_detection_results(self, detected_objects):
        self.results_text.config(state=tk.NORMAL)
        self.results_text.delete(1.0, tk.END)
        
        if not detected_objects:
            self.results_text.insert(tk.END, "No objects detected in current frame.")
            self.stats_label.config(text="No objects detected")
        else:
            total_objects = sum(detected_objects.values())
            self.stats_label.config(text=f"Total objects: {total_objects}")
            
            sorted_objects = sorted(detected_objects.items(), key=lambda x: x[1], reverse=True)
            self.results_text.insert(tk.END, "Detected Objects:\n\n")
            for obj, count in sorted_objects:
                self.results_text.insert(tk.END, f"• {obj}: {count}\n")
        
        self.results_text.config(state=tk.DISABLED)

    def stop_video(self):
        if not self.video_capture:
            return
            
        self.is_playing = False
        self.play_btn.config(text="Play")
        self.video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        self.show_current_frame()
        self.status_bar.config(text="Video stopped")

    def next_video(self):
        if not self.video_paths:
            return
            
        self.stop_video()
        self.current_video_index = (self.current_video_index + 1) % len(self.video_paths)
        self.load_video()
        self.status_bar.config(text=f"Switched to video {self.current_video_index + 1}/{len(self.video_paths)}")

    def toggle_detection(self):
        self.detection_enabled = not self.detection_enabled
        state_text = "ON" if self.detection_enabled else "OFF"
        self.detection_toggle.config(text=f"YOLO: {state_text}")
        
        if not self.detection_enabled:
            self.results_text.config(state=tk.NORMAL)
            self.results_text.delete(1.0, tk.END)
            self.results_text.insert(tk.END, "YOLO detection disabled")
            self.results_text.config(state=tk.DISABLED)
            self.stats_label.config(text="YOLO disabled")
        else:
            self.frame_count = self.detection_frequency - 1
            
        self.status_bar.config(text=f"YOLO Detection {state_text}")

    def toggle_face_detection(self):
        self.face_detection_enabled = not self.face_detection_enabled
        state_text = "ON" if self.face_detection_enabled else "OFF"
        self.face_toggle.config(text=f"Face: {state_text}")
        
        if not self.face_detection_enabled:
            if not self.detection_enabled:
                self.results_text.config(state=tk.NORMAL)
                self.results_text.delete(1.0, tk.END)
                self.results_text.insert(tk.END, "All detection disabled")
                self.results_text.config(state=tk.DISABLED)
                self.stats_label.config(text="All detection disabled")
        self.status_bar.config(text=f"Face Detection {state_text}")

    def show_current_frame(self):
        if not self.video_capture or not self.video_capture.isOpened():
            return
            
        current_pos = int(self.video_capture.get(cv2.CAP_PROP_POS_FRAMES))
        ret, frame = self.video_capture.read()
        
        if ret:
            detected_objects = {}
            if self.detection_enabled:
                frame, detected_objects = self.detect_objects(frame)
            if self.face_detection_enabled:
                frame, faces = self.detect_faces(frame)
                if faces:
                    detected_objects['face'] = faces
                    
            if detected_objects:
                self.update_detection_results(detected_objects)
                
            display_frame = self.resize_frame_to_fit(frame)
            display_frame = cv2.cvtColor(display_frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(display_frame)
            img = ImageTk.PhotoImage(image=img)
            
            self.video_frame.img = img
            self.video_frame.config(image=img)
        else:
            self.video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
            self.show_current_frame()

    def update_info_label(self):
        if not self.video_paths or not self.video_capture:
            self.info_label.config(text="No video loaded")
            return
            
        video_name = os.path.basename(self.video_paths[self.current_video_index])
        duration = self.total_frames / self.fps if self.fps > 0 else 0
        minutes, seconds = divmod(int(duration), 60)
        
        info_text = (
            f"Playing: {video_name} ({self.current_video_index + 1}/{len(self.video_paths)})\n"
            f"Resolution: {self.frame_width}x{self.frame_height} | "
            f"Duration: {minutes}:{seconds:02d} | "
            f"FPS: {self.fps:.1f}"
        )
        self.info_label.config(text=info_text)

    def on_closing(self):
        self.is_playing = False
        if self.video_capture:
            self.video_capture.release()
        self.root.destroy()

if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = PAUObjectDetectionApp(root)
        root.protocol("WM_DELETE_WINDOW", app.on_closing)
        root.mainloop()
    except Exception as e:
        print(f"Application error: {str(e)}")
        messagebox.showerror("Error", f"Application failed to start: {str(e)}")

CUDA not available, using CPU for YOLO
